# Import packages

In [30]:
import pandas as pd
import numpy as np

# Read Simulation Data

## Read joint state data

In [9]:
filePath = "simData"
fileName = "/plane_kp_100e10_kd_100/joint_state_test_plane_kp_100e10_kd_100_right_0.1.txt"
df = pd.read_csv(filePath+fileName, sep=" ", header=None)

In [10]:
df.head(80)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,611.6960-1.5114,-2.0393,1.0736,-0.0947,-2.7551,1.3077,1.5171,-1.9306,1.1701,-0.0052,...,-0.0043,0.0133,-0.0080,-0.0025,0.0018,0.0006,-0.0124,0.0213,-0.0130,NaN
1,611.7010-1.5114,-2.0393,1.0736,-0.0946,-2.7551,1.3077,1.5171,-1.9306,1.1701,-0.0052,...,-0.0154,0.0126,-0.0070,-0.0053,0.0021,-0.0009,-0.0163,-0.0035,-0.0003,NaN
2,611.7060-1.5114,-2.0393,1.0736,-0.0946,-2.7551,1.3077,1.5171,-1.9306,1.1701,-0.0051,...,-0.0199,0.0121,-0.0049,-0.0046,0.0026,0.0008,-0.0161,0.0210,-0.0140,NaN
3,611.7110-1.5114,-2.0393,1.0736,-0.0945,-2.7551,1.3077,1.5171,-1.9306,1.1701,-0.0051,...,-0.0318,0.0116,-0.0032,-0.0077,0.0030,-0.0004,-0.0261,-0.0015,-0.0027,NaN
4,611.7160-1.5114,-2.0393,1.0736,-0.0945,-2.7551,1.3077,1.5171,-1.9306,1.1701,-0.0051,...,-0.0338,0.0111,-0.0017,-0.0066,0.0033,0.0009,-0.0211,0.0217,-0.0152,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,612.0710-0.5916,-1.8188,0.8367,0.0063,-1.7967,0.6443,0.7283,-1.4035,0.6561,-0.0386,...,-0.0076,25.0000,0.9778,0.0000,0.6412,0.5901,0.0000,-0.2328,-0.4813,NaN
76,612.0760-0.5712,-1.8194,0.8380,0.0014,-1.8191,0.6501,0.7197,-1.4039,0.6549,-0.0279,...,0.0396,0.0000,1.3823,0.0000,1.3526,0.5945,0.0000,-0.1122,-0.7759,NaN
77,612.0810-0.5512,-1.8184,0.8381,-0.0004,-1.7991,0.6495,0.7105,-1.4052,0.6539,-0.0176,...,-0.1090,0.0000,2.1549,0.0000,1.9830,0.5725,0.0000,0.2184,-0.3582,NaN
78,612.0860-0.5310,-1.8172,0.8378,-0.0025,-1.7776,0.6500,0.7010,-1.4054,0.6531,-0.0081,...,-0.3105,0.0000,3.1104,0.0000,2.2437,0.6008,0.0000,-0.5548,0.1064,NaN


In [23]:
data = df.values
namesList = ['time', 'lf_hip_joint_position', 'lf_lower_leg_joint_position', 'lf_upper_leg_joint_position', 'lh_hip_joint_position', 'lh_lower_leg_joint_position', 'lh_upper_leg_joint_position',
            'rf_hip_joint_position', 'rf_lower_leg_joint_position', 'rf_upper_leg_joint_position', 'rh_hip_joint_position', 'rh_lower_leg_joint_position', 'rh_upper_leg_joint_position',
            'lf_hip_joint_vel', 'lf_lower_leg_joint_vel', 'lf_upper_leg_joint_vel', 'lh_hip_joint_vel', 'lh_lower_leg_joint_vel', 'lh_upper_leg_joint_vel',
            'rf_hip_joint_vel', 'rf_lower_leg_joint_vel', 'rf_upper_leg_joint_vel', 'rh_hip_joint_vel', 'rh_lower_leg_joint_vel', 'rh_upper_leg_joint_vel',
            'lf_hip_joint_effort', 'lf_lower_leg_joint_effort', 'lf_upper_leg_joint_effort', 'lh_hip_joint_effort', 'lh_lower_leg_joint_effort', 'lh_upper_leg_joint_effort',
            'rf_hip_joint_effort', 'rf_lower_leg_joint_effort', 'rf_upper_leg_joint_effort', 'rh_hip_joint_effort', 'rh_lower_leg_joint_effort', 'rh_upper_leg_joint_effort']

In [24]:
staticCounter = 0
for i in range(2):
    if staticCounter == 0:
        print(i)
        i -= 1
        staticCounter += 1
    else:
        print(i)

0
1


In [32]:
data[:,0][0:8]

array(['611.6960-1.5114', '611.7010-1.5114', '611.7060-1.5114',
       '611.7110-1.5114', '611.7160-1.5114', '611.7210-1.5114',
       '611.7260-1.5114', '611.7310-1.5114'], dtype=object)

In [31]:
dictCategory = {}
staticCounter = 0 # Variable to account for formatting issue
for index in range(len(namesList)-1):
    if index == 0 and staticCounter == 0: # Condition for formatting issue when collecting data
        dictCategory[namesList[index]] = data[:,index][0:8].astype(float)
        dictCategory[namesList[index+1]] = data[:,index][8:].astype(float)
    else:
        dictCategory[namesList[index+1]] = float(data[index]).astype(float)

ValueError: could not convert string to float: '611.6960-1.5114'

## Select joint states to transform to matrix

In [ ]:
desiredJoints = ['lf_hip_joint_position', 'lf_hip_joint_vel']
